In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd

from glob import glob
from os.path import isfile
from functools import partial
from subprocess import Popen, call
from multiprocessing import get_context

os.environ['OMP_NUM_THREADS'] = '1'

In [2]:
script_dir = '/uufs/chpc.utah.edu/common/home/u1070830/code/snow-liquid-ratio/'
era5_script_dir = '/uufs/chpc.utah.edu/common/home/u1070830/code/model-tools/era5/'
obs_path = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/observations/'
era5_path = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/era5/'
gfs_path = '/uufs/chpc.utah.edu/common/home/steenburgh-group10/mewessler/gfs/'

## Start with the metadata file as an efficent way to keep data organized vs a config file

In [8]:
metadata = pd.read_excel(obs_path + 'Dataset_Metadata.xlsx')

# Temp fix to use CLNX instead of CLN data
metadata['code'] = metadata['code'].replace('CLN', 'CLNX')
metadata = metadata.set_index('code')

metadata.iloc[:, :-1]

,name,owner,city,state,lat,lon,elevation_m,start,end,interval,n_events,auto_manual,snow_type,swe_type
code,,,,,,,,,,,,,,
CLNX,Alta Collins,Alta Ski Lifts,Alta,UT,40.576300,-111.638300,2945,NaN,NaN,12/24,NaN,Manual,Snow Board (HSN),NaN
AGD,Alta Guard,Utah Dept. of Transportation,Alta,UT,40.590500,-111.638000,2682,NaN,NaN,6/12/24,NaN,Manual,Snow Board (HSN),NaN
ALTA,Alta CoOp,National Weather Service,Alta,UT,40.590580,-111.637030,2655,NaN,NaN,12/24,NaN,Manual,NaN,NaN
BCC,Big Cottonwood,Utah Dept. of Transportation,Brighton,UT,40.650300,-111.649900,2224,NaN,NaN,12/24,NaN,Manual,NaN,NaN
PVC,Provo Canyon,Utah Dept. of Transportation,Provo,UT,40.334100,-111.613300,1560,NaN,NaN,12/24,NaN,Manual,NaN,NaN
SLB,Silver Lake Brighton,National Weather Service,Brighton,UT,40.600800,-111.584200,2664,NaN,NaN,24,NaN,Manual,NaN,NaN
BSNFDC,Devil's Creek,Burlington Northern Santa Fe Corp,NaN,MT,48.277725,-113.432578,1415,NaN,NaN,NaN,NaN,Manual,NaN,NaN
BSNFEX,Essex,Burlington Northern Santa Fe Corp,Essex,MT,48.281760,-113.606570,1173,NaN,NaN,NaN,NaN,Manual,NaN,NaN
BSNFJE,Java East,Burlington Northern Santa Fe Corp,NaN,MT,48.235870,-113.564530,1232,NaN,NaN,NaN,NaN,Manual,NaN,NaN


## Loop over each station:
We don't want to re-run on data already processed, so check each step first

- Ensure the observation data exists and is read in to .pd

In [9]:
site_list = metadata.index.values.astype('str')
print('Sites to process:', site_list)

ingest_script = script_dir + 'data-tools/generic_data_ingest.py'

for site in site_list:
    
    if len(glob(obs_path + 'clean/%s_*.pd'%site)) > 0:
        print('File exists, skipping: %s'%site)
    else:
        try:
            run_cmd = ('python ' + ingest_script + ' %s'%site)
            call(run_cmd, shell=True)

        except:
            print('Ingest %s failed to run'%site)
            # raise

        else:
            # Check to see if file was written before declaring success - 
            # call won't catch failures in the child script
            
            if len(glob(obs_path + 'clean/%s_*.pd'%site)) > 0:
                print('Ingest %s success'%site)
                
            else:
                print('Ingest %s failed to write'%site)
                # raise
                
                # Fail cases (log these within child script:
                #     No header info for site/file
                #     No file found for site
                #     Header mismatch/read error
                #     Data mismatch error (automated - manual too large)

Sites to process: ['CLNX' 'AGD' 'ALTA' 'BCC' 'PVC' 'SLB' 'BSNFDC' 'BSNFEX' 'BSNFJE']
File exists, skipping: CLNX
File exists, skipping: AGD
File exists, skipping: ALTA
File exists, skipping: BCC
File exists, skipping: PVC
File exists, skipping: SLB
File exists, skipping: BSNFDC
File exists, skipping: BSNFEX
File exists, skipping: BSNFJE


- Ensure the era5 profile has been created

In [10]:
# for site in site_list:
def profiles(site, metadata):

    site_lat, site_lon = metadata.loc[site, ['lat', 'lon']].values
    
    # We need to first determine the output filename (All ERA5 profiles are xx.xxN, xxx.xxW)
    sample = xr.open_dataset(era5_script_dir + 'era5_sample.nc')
    a = abs(sample['latitude']-site_lat)+abs(sample['longitude']-360-site_lon)
    yi, xi = np.unravel_index(a.argmin(), a.shape)

    lat = sample.isel(latitude=yi, longitude=xi)['latitude']
    lon = sample.isel(latitude=yi, longitude=xi)['longitude'] - 360
    
    era5_prof_file = 'era5prof_%.2fN_%.2fW.nc'%(lat, abs(lon))

    print('\nSite: %s %.3f %.3f\n%s\n'%(site, site_lat, site_lon, era5_prof_file))
    
    iter_count = 0
    while not isfile(era5_path + '/profiles/' + era5_prof_file):
        iter_count += 1
        
        run_cmd = era5_script_dir + 'extract_agg_profile.sh %.2f %.2f 1980 2020'%(lat, lon)
        P = Popen(run_cmd, shell=True, stdout=PIPE, stderr=PIPE)
        output, err = P.communicate()

    print('%s Complete'%era5_prof_file)

# 10 process workers seems OK for now...
worker_cap = 10
n_workers = len(site_list) if len(site_list) < worker_cap else worker_cap

profiles_mp = partial(profiles, metadata=metadata)
with get_context('fork').Pool(10) as p:
    p.map(profiles_mp, site_list)
    p.close()
    p.join()


Site: BSNFEX 48.282 -113.607
era5prof_48.25N_113.50W.nc

Site: BCC 40.650 -111.650
era5prof_40.75N_111.75W.nc

Site: AGD 40.590 -111.638
era5prof_40.50N_111.75W.nc

Site: BSNFDC 48.278 -113.433
era5prof_48.25N_113.50W.nc

Site: ALTA 40.591 -111.637
era5prof_40.50N_111.75W.nc

Site: SLB 40.601 -111.584
era5prof_40.50N_111.50W.nc

Site: CLNX 40.576 -111.638
era5prof_40.50N_111.75W.nc

Site: PVC 40.334 -111.613
era5prof_40.25N_111.50W.nc

Site: BSNFJE 48.236 -113.565
era5prof_48.25N_113.50W.nc









era5prof_40.50N_111.50W.nc Complete
era5prof_40.25N_111.50W.nc Complete
era5prof_48.25N_113.50W.nc Complete
era5prof_40.50N_111.75W.nc Complete
era5prof_40.75N_111.75W.nc Complete
era5prof_48.25N_113.50W.nc Complete
era5prof_40.50N_111.75W.nc Complete
era5prof_40.50N_111.75W.nc Complete
era5prof_48.25N_113.50W.nc Complete


- Pair the observations and ERA5 profile

In [ ]:
# Pass cases:
#     Produced new site pair
#     Site pair found

- Extract a GFS profile for verification

In [ ]:
# Pass cases:
#     Produced new GFS profile
#     GFS profile found

In [ ]:
print('Data Pre-Processing Completed...')